#   Bibliotecas

- Versões das bibliotecas no README

In [1]:
import pandas as pd
import random 
import numpy as np
import re
from scipy.sparse import hstack #utilizada para a junção de matrizes

In [2]:
#Pré processamento de colunas

from sklearn.model_selection import train_test_split #Divisão do dataset entre treino e teste
from sklearn.feature_extraction.text import TfidfVectorizer #vetorização do dataset
from sklearn.preprocessing import OneHotEncoder #aplicação de dummy

In [3]:
#Pré processamento de texto


import nltk
from nltk import word_tokenize #tokenizador
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords   #stopword

nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
#import de funções do script de pre_processamento

from pre_processamento import pre_processamento, text_preprocess, textcolumns_junct,stemming

In [5]:
# Modelo a ser testado
import tensorflow as tf
from keras import layers
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.metrics import RootMeanSquaredError

In [6]:
#metricas

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error

In [7]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9340559379129461765
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2252026676
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16652033083150953189
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


#   Dados

In [8]:
train = pd.read_csv(r"C:\Users\angel\Desktop\bootcamp\train.tsv", sep='\t')
test = pd.read_csv(r"C:\Users\angel\Desktop\bootcamp\test_stg2.tsv", sep='\t')

In [9]:
train,test = pre_processamento(train,test)

#   Split

In [10]:
#Separação em treino e teste para começarmos as transformações, protegendo nossos dados de teste de possíveis vazamentos e das mudanças de hyperparametros. 

X1, Xtest = train_test_split(train, test_size=0.1, random_state=10) #Divisão em 90/10

In [11]:
#Separação em treino e teste para começarmos as transformaçõs, protegendo nossos dados de validação de possíveis vazamentos. 

Xtrain, Xval = train_test_split(X1, test_size=0.2, random_state=10) #Divisão em 80/20

In [12]:
print(Xtrain.shape) 
print(Xval.shape)
print(Xtest.shape)

(1066795, 13)
(266699, 13)
(148167, 13)


#   Pré-processamento 

In [13]:
#Função pra juntar as colunas de texto em uma só coluna

Xtrain = textcolumns_junct(Xtrain) 
Xval = textcolumns_junct(Xval) 
Xtest = textcolumns_junct(Xtest) 

In [14]:
#Convertendo tudo para string

Xtrain["item_description"] = Xtrain["item_description"].astype(str)
Xval["item_description"] = Xval["item_description"].astype(str)
Xtest["item_description"] = Xtest["item_description"].astype(str)

In [15]:
#Limpando texto com regex.
Xtrain["item_description"] = Xtrain["item_description"].apply(lambda x: text_preprocess(x))
Xval["item_description"] = Xval["item_description"].apply(lambda x: text_preprocess(x))
Xtest["item_description"] = Xtest["item_description"].apply(lambda x: text_preprocess(x))

In [16]:
#tokenizando

Xtrain["item_description"] = Xtrain["item_description"].apply(lambda x: word_tokenize(x))
Xval["item_description"] = Xval["item_description"].apply(lambda x: word_tokenize(x))
Xtest["item_description"] = Xtest["item_description"].apply(lambda x: word_tokenize(x))

In [17]:
Xtrain["item_description"] = Xtrain["item_description"].apply(lambda x: stemming(x))
Xval["item_description"] = Xval["item_description"].apply(lambda x: stemming(x))
Xtest["item_description"] = Xtest["item_description"].apply(lambda x: stemming(x))

In [18]:
Xtrain["item_description"]=[" ".join(review) for review in Xtrain["item_description"].values]
Xval["item_description"]=[" ".join(review) for review in Xval["item_description"].values]
Xtest["item_description"]=[" ".join(review) for review in Xtest["item_description"].values]

In [19]:
train["item_description"]

0                                         No description yet
1          This keyboard is in great condition and works ...
2          Adorable top with a hint of lace and a key hol...
3          New with tags. Leather horses. Retail for [rm]...
4                  Complete with certificate of authenticity
                                 ...                        
1482530    Lace, says size small but fits medium perfectl...
1482531     Little mermaid handmade dress never worn size 2t
1482532            Used once or twice, still in great shape.
1482533    There is 2 of each one that you see! So 2 red ...
1482534    New with tag, red with sparkle. Firm price, no...
Name: item_description, Length: 1481661, dtype: object

In [25]:
#Xtrain.to_pickle(r"C:\Users\angel\Desktop\bootcamp\Xtrain.pkl")
#Xval.to_pickle(r"C:\Users\angel\Desktop\bootcamp\Xval.pkl")
#Xtest.to_pickle(r"C:\Users\angel\Desktop\bootcamp\Xtest.pkl")

In [21]:
#treinando o modelo com valores até $250, que representam 89% do dataset
Xtrain = Xtrain.drop(Xtrain[~((Xtrain['price'] > 0) & (Xtrain['price'] <= 250))].index)

In [22]:
X = Xtrain.drop("price", axis = 1)
y = Xtrain["price"]

In [23]:
X = Xval.drop("price", axis = 1)
y = Xval["price"]

In [24]:
X = Xtest.drop("price", axis = 1)
y = Xtest["price"]

# Transformadores